In [1]:
import gradio as gr
from config import CONFIG
from skimmer import PDFSkimmer
from chatbot import Chatbot

BASE_DIR = CONFIG["directories"]["base_dir"]
BASE_URL = CONFIG["directories"]["base_url"]


In [2]:
skimmer = PDFSkimmer()
chatbot = Chatbot(type="analysis")


def get_summary(file_path: str):
    return skimmer.load_or_summary(file_path)


In [3]:
file_path = "/Volumes/AnselmJeong.ddns.net/LIBRARY/Articles/d41586-025-00068-5.pdf"

In [4]:
response = get_summary(file_path)
print(response)

No summary found for /Volumes/AnselmJeong.ddns.net/LIBRARY/Articles/d41586-025-00068-5.pdf
URL: http://anselmjeong.synology.me:8091/share/GXzzOcfK/d41586-025-00068-5.pdf
Response from the LLM:

{'core_question': 'What is the core research question?', 'introduction': 'What is the background and context of the study?', 'methodology': 'What is the methodology of the study?', 'results': 'What are the key results and findings?', 'discussion': 'What are the implications of the results?', 'limitations': 'What are the limitations of the study?'}


TypeError: custom_types.DatabaseSummary() argument after ** must be a mapping, not PaperSummary

In [ ]:
from article_analysis import ArticleAnalyzer

AA = ArticleAnalyzer()
response = AA.summarize(file_path)


In [5]:
chatbot.set_url("http://anselmjeong.synology.me:8091/share/GXzzOcfK/d41586-025-00068-5.pdf")

In [ ]:
chatbot.query_llm()

In [ ]:
chatbot.history_langchain

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            file_explorer = gr.FileExplorer(glob="**/*.pdf", root_dir=BASE_DIR, file_count="single")
        with gr.Column():
            url = gr.Textbox(label="URL")
            path = gr.Textbox(label="File Path")
            result = gr.JSON(label="Summary")

            chat_interface = gr.ChatInterface(
                fn=get_summary,
                type="messages",
                title="PDF Skimmer",
                description="Summarize a PDF file.",
            )

    file_explorer.change(fn=get_summary, inputs=file_explorer, outputs=[result])


demo.launch()
